# GROUP BY, HAVING

In [86]:
import sqlite3 as lite
import pandas as pd

In [87]:
db = lite.connect('./products.db')

In [88]:
pd.options.display.max_colwidth = 200

## To-do
- Get the total sales, total revenue, and the average price per month on orders whose quantity > 20

In [89]:
# Inspect tables, schema
query = """
SELECT name, sql 
FROM sqlite_master
;
"""

pd.read_sql(query, db)

,name,sql
0,Categories,"CREATE TABLE Categories(\n ""CategoryID"" TEXT,\n ""CategoryName"" TEXT,\n ""Description"" TEXT\n)"
1,Customers,"CREATE TABLE Customers(\n ""CustomerID"" TEXT,\n ""CustomerName"" TEXT,\n ""ContactName"" TEXT,\n ""Address"" TEXT,\n ""City"" TEXT,\n ""PostalCode"" TEXT,\n ""Country"" TEXT\n)"
2,Employees,"CREATE TABLE Employees(\n ""EmployeeID"" TEXT,\n ""LastName"" TEXT,\n ""FirstName"" TEXT,\n ""BirthDate"" TEXT,\n ""Photo"" TEXT,\n ""Notes"" TEXT\n)"
3,Shippers,"CREATE TABLE Shippers(\n ""ShipperID"" TEXT,\n ""ShipperName"" TEXT,\n ""Phone"" TEXT\n)"
4,Suppliers,"CREATE TABLE Suppliers(\n ""SupplierID"" TEXT,\n ""SupplierName"" TEXT,\n ""ContactName"" TEXT,\n ""Address"" TEXT,\n ""City"" TEXT,\n ""PostalCode"" TEXT,\n ""Country"" TEXT,\n ""Phone"" TEXT\n)"
5,Orders,"CREATE TABLE Orders(\nOrderID integer,\nCustomerID integer,\nEmployeeID integer,\nOrderDate date,\nShipperID integer)"
6,OrderDetails,"CREATE TABLE OrderDetails(\nOrderDetailID integer,\nOrderID integer,\nProductID integer,\nQuantity integer)"
7,Products,"CREATE TABLE Products(\nProductID integer,\nProductName text,\nSupplierID integer,\nCategoryID integer,\nUnit text,\nPrice integer\n)"


In [90]:
#join the tables on the requirement
query = """
SELECT *
FROM OrderDetails D
    JOIN
        Products P
        ON
            P.ProductID = D.ProductID
    JOIN 
        Orders O
        ON 
            O.OrderID = D.OrderID   
;
"""

pd.read_sql(query, db).head()

,OrderDetailID,OrderID,ProductID,Quantity,ProductID,ProductName,SupplierID,CategoryID,Unit,Price,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,1,10248,11,12,11,Queso Cabrales,5,4,1 kg pkg.,21.00,10248,90,5,1996-07-04,3
1,2,10248,42,10,42,Singaporean Hokkien Fried Mee,20,5,32 - 1 kg pkgs.,14.00,10248,90,5,1996-07-04,3
2,3,10248,72,5,72,Mozzarella di Giovanni,14,4,24 - 200 g pkgs.,34.80,10248,90,5,1996-07-04,3
3,4,10249,14,9,14,Tofu,6,7,40 - 100 g pkgs.,23.25,10249,81,6,1996-07-05,1
4,5,10249,51,40,51,Manjimup Dried Apples,24,7,50 - 300 g pkgs.,53.00,10249,81,6,1996-07-05,1


In [91]:
# select the required data ( D.Quantity, P.Price, O.OrderDate)
query = """
SELECT D.Quantity, P.Price, O.OrderDate
FROM OrderDetails D
    JOIN
        Products P
        ON
            P.ProductID = D.ProductID
    JOIN 
        Orders O
        ON 
            O.OrderID = D.OrderID   
;
"""

pd.read_sql(query, db).head()

,Quantity,Price,OrderDate
0,12,21.00,1996-07-04
1,10,14.00,1996-07-04
2,5,34.80,1996-07-04
3,9,23.25,1996-07-05
4,40,53.00,1996-07-05


In [92]:
# add a new column named "Month" using substr
# index in SQL starts from 1, not 0

query = """
SELECT substr(O.OrderDate, 1, 7) "Month", D.Quantity, P.Price, O.OrderDate
FROM OrderDetails D
    JOIN
        Products P
        ON
            P.ProductID = D.ProductID
    JOIN 
        Orders O
        ON 
            O.OrderID = D.OrderID   
;
"""

pd.read_sql(query, db).head()

,Month,Quantity,Price,OrderDate
0,1996-07,12,21.00,1996-07-04
1,1996-07,10,14.00,1996-07-04
2,1996-07,5,34.80,1996-07-04
3,1996-07,9,23.25,1996-07-05
4,1996-07,40,53.00,1996-07-05


In [13]:
#group by Month

query = """
SELECT substr(O.OrderDate, 1, 7) "Month", D.Quantity, P.Price, O.OrderDate
FROM OrderDetails D
    JOIN
        Products P
        ON
            P.ProductID = D.ProductID
    JOIN 
        Orders O
        ON 
            O.OrderID = D.OrderID   
    GROUP BY 
        substr(O.OrderDate, 1, 7)
;
"""

pd.read_sql(query, db).head()

,Month,Quantity,Price,OrderDate
0,1996-07,20,34.80,1996-07-31
1,1996-08,6,7.75,1996-08-30
2,1996-09,20,18.00,1996-09-30
3,1996-10,15,18.00,1996-10-31
4,1996-11,35,33.25,1996-11-29


In [17]:
# select required data using aggregation

query = """
SELECT 
    substr(O.OrderDate, 1, 7) "Month", D.Quantity, P.Price, O.OrderDate,
    SUM(D.Quantity) "SalesCount", 
    SUM(D.Quantity * P.Price) "Revenue",
    ROUND(AVG(D.Quantity * P.Price), 2) "Average"
FROM OrderDetails D
    JOIN
        Products P
        ON
            P.ProductID = D.ProductID
    JOIN 
        Orders O
        ON 
            O.OrderID = D.OrderID   
    GROUP BY 
        substr(O.OrderDate, 1, 7)
;
"""

pd.read_sql(query, db).head()

,Month,Quantity,Price,OrderDate,SalesCount,Revenue,Average
0,1996-07,20,34.80,1996-07-31,1462,37779.85,640.34
1,1996-08,6,7.75,1996-08-30,1322,33285.49,482.40
2,1996-09,20,18.00,1996-09-30,1124,34565.60,606.41
3,1996-10,15,18.00,1996-10-31,1738,51528.69,705.87
4,1996-11,35,33.25,1996-11-29,1735,62163.99,941.88


In [93]:
# add the condition "Quantity > 20" to GROUP BY using HAVING
#final answer

query = """
SELECT 
    substr(O.OrderDate, 1, 7) "Month", D.Quantity, P.Price, O.OrderDate,
    SUM(D.Quantity) "SalesCount", 
    SUM(D.Quantity * P.Price) "Revenue",
    ROUND(AVG(D.Quantity * P.Price), 2) "Average"
FROM OrderDetails D
    JOIN
        Products P
        ON
            P.ProductID = D.ProductID
    JOIN 
        Orders O
        ON 
            O.OrderID = D.OrderID   
    GROUP BY 
        substr(O.OrderDate, 1, 7)
    HAVING 
        D.Quantity > 20
;
"""

pd.read_sql(query, db)

,Month,Quantity,Price,OrderDate,SalesCount,Revenue,Average
0,1996-11,35,33.25,1996-11-29,1735,62163.99,941.88
1,1997-01,40,7.45,1997-01-31,2401,83400.47,981.18


# NoSQL
- Schemaless <br/>
- JOIN impossible <br/>

## Types of NoSQL
- {Key:Value} : Redis <br/>
- [Column] : Cassandra, HBase <br/>
- Document {Key:{Key:Value}} = CouchDB, MongoDB <br/>

## MongoDB
- BSON(Binary JSON) based Key-Value Store<br/>
- JSON type documents <br/>
- Collection -> Document -> Key:Value Data

### Requirements
- DB instance(mLab) <br/>
- pymongo (pip install pymongo) <br/>
- pandas (pip install pandas) <br/>
- requests (pip install requests) <br/>
- jupyter notebook (pip install jupyter) <br/>

# MongoDB with Jupyter Notebook

In [21]:
import requests
from pymongo import MongoClient

In [20]:
mongo_uri = "mongodb://<mLabusername>:<mLabpassword>@ds145299.mlab.com:45299/mydbinstance"

In [22]:
client = MongoClient(mongo_uri) 

In [24]:
client.mydbinstance.collection_names()

c:\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['system.indexes', 'users']

In [25]:
db = client.mydbinstance

In [26]:
db.users

Collection(Database(MongoClient(host=['ds145299.mlab.com:45299'], document_class=dict, tz_aware=False, connect=True), 'mydbinstance'), 'users')

In [27]:
db["users"]

Collection(Database(MongoClient(host=['ds145299.mlab.com:45299'], document_class=dict, tz_aware=False, connect=True), 'mydbinstance'), 'users')

In [29]:
users_collection = db.users

## INSERT : insert_one(), insert_many()

In [30]:
some_user = {
    "name" : "Fastcampus Kim",
    "email" : "fckim@fastcampus.co.kr",
}

users_collection.insert_one(some_user)

In [32]:
lots_of_users = [
    {
        "name": "jyp",
        "email": "jyp@fastcampus.co.kr"
     }, 
    {
        "name" : "gd",
        "address" : "Seoul, Korea"
    }, 
    {
        "name" : "Sbucks"
    }]

users_collection.insert_many(lots_of_users)

## SELECT : find_one(), find()

In [33]:
query = {}
users_collection.find_one(query)

{'_id': ObjectId('5c398d87a6bb5b6cc4db1d32'),
 'name': 'Fastcampus Kim',
 'email': 'fckim@fastcampus.co.kr'}

In [35]:
query = {}
list(users_collection.find(query))

[{'_id': ObjectId('5c398d87a6bb5b6cc4db1d32'),
  'name': 'Fastcampus Kim',
  'email': 'fckim@fastcampus.co.kr'},
 {'_id': ObjectId('5c399361a6bb5b6cc4db1d33'),
  'name': 'jyp',
  'email': 'jyp@fastcampus.co.kr'},
 {'_id': ObjectId('5c399361a6bb5b6cc4db1d34'),
  'name': 'gd',
  'address': 'Seoul, Korea'},
 {'_id': ObjectId('5c399361a6bb5b6cc4db1d35'), 'name': 'Sbucks'}]

In [36]:
#same result as above
[item for item in users_collection.find(query)]

[{'_id': ObjectId('5c398d87a6bb5b6cc4db1d32'),
  'name': 'Fastcampus Kim',
  'email': 'fckim@fastcampus.co.kr'},
 {'_id': ObjectId('5c399361a6bb5b6cc4db1d33'),
  'name': 'jyp',
  'email': 'jyp@fastcampus.co.kr'},
 {'_id': ObjectId('5c399361a6bb5b6cc4db1d34'),
  'name': 'gd',
  'address': 'Seoul, Korea'},
 {'_id': ObjectId('5c399361a6bb5b6cc4db1d35'), 'name': 'Sbucks'}]

In [37]:
[item["name"] for item in users_collection.find(query)]

['Fastcampus Kim', 'jyp', 'gd', 'Sbucks']

In [38]:
# SELECT the data WHERE name = "jyp"
query = {
    "name": "jyp",
}
list(users_collection.find(query))

[{'_id': ObjectId('5c399361a6bb5b6cc4db1d33'),
  'name': 'jyp',
  'email': 'jyp@fastcampus.co.kr'}]

<img src="./img/nosql_operator.png" width="800" style="float: left">

In [40]:
# SELECT the data WHERE name in ["jyp", "gd"]
query = {
    "name": {
        "$in": ["jyp", "gd"]
    }
}
list(users_collection.find(query))

[{'_id': ObjectId('5c399361a6bb5b6cc4db1d33'),
  'name': 'jyp',
  'email': 'jyp@fastcampus.co.kr'},
 {'_id': ObjectId('5c399361a6bb5b6cc4db1d34'),
  'name': 'gd',
  'address': 'Seoul, Korea'}]

## Zigbang example

In [41]:
bigbang_collection = db.bigbang

In [49]:
url = "https://apis.zigbang.com/v3/items?detail=true&item_ids=[14388831,14284983,14538654,14187437,14359880,14398188,14399002,14454456,14398156,14398265,14455800,14360392,14472148,14507858,14454383,14511262,14407326,14470986,14398229,14455772,14397994,14164631,14317786,14271650,14359830,14512053,14455741,14062510,14324683,14269988,14169764,14328611,14536531,14321635,14398108,14467241,14448306,14226360,14467013,14317887,14513370,13416109,14165519,14263914,14222733,14254448,14203858,14328661,14241599,14407024,14138253,14527248,14175178,14147978,14178200,14218025,14285394,14219393,14210656,14258795]"
requests.get(url)

<Response [200]>

In [50]:
data = requests.get(url).json()['items']

In [51]:
bigbang_collection.insert_many(data)

bigbang_collection.find({}).count()

c:\python37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


60

In [52]:
# Inspect the first json data using find_one()
bigbang_collection.find_one({})

{'_id': ObjectId('5c399b2ea6bb5b6cc4db1d39'),
 'title': '세종시 조치원읍 침산리',
 'header': False,
 'header_height': 0,
 'item': {'id': 14187437,
  'images': [{'index': 0,
    'count': 1,
    'url': 'https://ic.zigbang.com/ic/items/14187437/1.jpg'},
   {'index': 1,
    'count': 2,
    'url': 'https://ic.zigbang.com/ic/items/14187437/2.jpg'},
   {'index': 2,
    'count': 3,
    'url': 'https://ic.zigbang.com/ic/items/14187437/3.jpg'},
   {'index': 3,
    'count': 4,
    'url': 'https://ic.zigbang.com/ic/items/14187437/4.jpg'},
   {'index': 4,
    'count': 5,
    'url': 'https://ic.zigbang.com/ic/items/14187437/5.jpg'},
   {'index': 5,
    'count': 6,
    'url': 'https://ic.zigbang.com/ic/items/14187437/6.jpg'},
   {'index': 6,
    'count': 7,
    'url': 'https://ic.zigbang.com/ic/items/14187437/7.jpg'}],
  'is_realestate': True,
  'is_direct': False,
  'is_room': False,
  'is_type_room': False,
  'rent': 23,
  'deposit': 100,
  'is_deposit_only': False,
  'floor': '3층',
  '_floor': '3층',
  'floo

In [54]:
# get rent 
query = {}

bigbang_collection.find_one(query)["item"]["rent"]

23

In [59]:
# get deposit
query = {}

bigbang_collection.find_one(query)["item"]["deposit"]

100

In [68]:
# get items where 100<= deposit <= 2000 and rent <= 50 from bigbang_collection
query = {
    "$or": [
        {
            "item.deposit":{
                "$lte":2000,
                "$gte":100,
            }
        },
        {
            "item.rent":{
                "$lte":50,
            }
        }
    ]
}

len(list(bigbang_collection.find(query)))

55

## Naver realtime keyword example

In [82]:
from bs4 import BeautifulSoup
from time import ctime

In [83]:
html = requests.get('https://www.naver.com/').text
executed_time = ctime() #value1

soup = BeautifulSoup(html, 'html.parser')
kw_list = soup.find("ul", attrs={"class": "ah_l"})
all_li = kw_list.find_all("span", attrs={"class":"ah_k"})
result = [] #value2
for li in all_li:
    result.append(li.text)
    
result
#result = ['','','']
#{"executed": "2019-01-12 17:25:00",
# "keywords": ["스카이캐슬..", ""]}

nv_collection = db.nvkwlist

data = {
    "executed": executed_time,
    "keywords": result,
}

nv_collection.insert_one(data)

In [84]:
nv_collection.find_one({})

{'_id': ObjectId('5c39a65da6bb5b6cc4db1d72'),
 'executed': 'Sat Jan 12 17:33:33 2019',
 'keywords': ['스카이캐슬 스포',
  '주진모',
  '몽키킹3',
  '어뜨무러차',
  '박소연',
  '베트남 이란',
  '음악중심',
  '자이글 롤링쿡스',
  '모아나',
  '청파동 냉면집',
  '만물상아귀찜',
  '신애라',
  '허안나',
  '황교안',
  '미세먼지 비상저감조치',
  '한국 중국',
  '케어',
  '말리와 나',
  'sky 캐슬 스포',
  '쇼음악중심']}